<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr

In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))
  print('-------------------')
  print('Est LTT: ',  format_amt(estimate_lt))
  print('Est LTT PnL: '+ '{} ({}%)'.format(format_amt(est_lt_pnl_amount), est_lt_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'TFrame', 'LatestQtr', 'StarStock', 'BizFinTech', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')

Run date time (IST): 2025-01-15 13:45:03
-------------------
qualified stocks: 88
with latest results: 83
still star stocks: 43
-------------------
Initial Investment:  1.04 C
CY Investment:  1.17 C
Reserve:  0.00
Current:  1.15 C
-------------------
Today PnL: -11.15 K (-0.1%)
Current PnL: -14.79 L (-12.6%)
CY Booked + Current PnL: -2.89 L (-2.46%)
-------------------
Total profit:  2.93 L
Total loss:  -17.72 L
-------------------
Total Booked + Current PnL: 10.22 L (9.8%)
Total Booked PnL: 25.01 L (23.99%)
Curr Year Booked PnL: 11.90 L (10.38%)
Prev Year Booked PnL: 13.11 L (12.57%)
Est FTT:  1.87 C
Est FTT PnL: 72.38 L (63.14%)
-------------------
Est LTT:  2.48 C
Est LTT PnL: 1.33 C (116.47%)
Deployed:  1.04 C
Current:  1.15 C
CAGR/XIRR %: 8.01%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'AvgCost', 'FTT', 'Dev%_PE', 'Conviction', 'RSP', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'XIRR', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

Symbol  AvgCost     FTT  Dev%_PE Conviction    RSP   Current  \
14   BAYERCROP  4346.57  6805.0    12.32       H-SC  11.36   83414.0   
79         TCS  4164.02  4998.0     3.97       X-LC  72.73  164666.0   
72         SIS   477.00   528.0    56.94       M-SC  44.32   56610.0   
40        INFY  1436.24  2275.0     8.81       X-LC  87.50  202758.0   
47  LALPATHLAB  2630.06  3545.0   -19.93       H-SC  60.23  121638.0   

    Current P/L  FTT Amt  Today P/L%  Current P/L%   FTT%   OTT%  XIRR  \
14      13869.0  25466.0        0.70         19.94  30.53  56.56  12.0   
79       2269.0  30249.0       -0.26          1.40  18.37  20.03  15.0   
72     -24480.0  33151.0        0.62        -30.19  58.56  10.69 -27.0   
40      53389.0  33840.0        0.49         35.74  16.69  58.40  37.0   
47       5915.0  34338.0       -1.38          5.11  28.23  34.79   9.0   

    RRR Ind  CurrAlloc%  Gained% Criteria Strategy    Category  
14     0.54        0.73     6.79     X40N      NTT   CHEMICALS  
79     0.08        1.44    15.56      X40      BTT          IT  
72    -0.74        0.49     0.62     X40N      NTT        MISC  
40     1.58        1.77    41.48      X40      BTT          IT  
47     0.17        1.06    38.28     X40N      NTT  HEALTHCARE

In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

Symbol  AvgCost       FTT  Dev%_PE Conviction    RSP   Current  \
1     AARTIIND   487.04    919.00    -2.77       M-SC  81.82   71309.0   
31  HAPPSTMNDS   919.80   1491.84   -14.87       M-SC  85.23   76408.0   
58       QUESS   711.66    986.00   -27.80       M-SC  71.59  139842.0   
19        CERA  9059.90  10420.45   -11.94       X-SC  35.23   68227.0   
46   KOTAKBANK  1767.52   2270.00   -43.10       X-LC  89.77  283497.0   

    Current P/L  FTT Amt  Today P/L%  Current P/L%    FTT%   OTT%  XIRR  \
1      -10027.0  82162.0        4.31        -12.33  115.22  88.69 -11.0   
31     -20171.0  80236.0        2.10        -20.89  105.01  62.19 -21.0   
58     -16012.0  76088.0        2.07        -10.27   54.41  38.55 -25.0   
19     -22372.0  35976.0        1.87        -24.69   52.73  15.02 -54.0   
46       2461.0  77423.0        1.85          0.88   27.31  28.43  -0.0   

    RRR Ind  CurrAlloc%  Gained% Criteria Strategy   Category  
1     -0.12        0.62     7.33       XR      NTT  CHEMICALS  
31    -0.25        0.67     4.00       AR      ATH         IT  
58    -0.21        1.22    36.57     XY24      NTT       MISC  
19    -0.62        0.60     3.58     X40N      ATH   CERAMICS  
46     0.03        2.47    15.40      X40      BTT      BANKS

In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

Symbol  AvgCost     FTT  Dev%_PE Conviction    RSP   Current  \
9   BAJAJFINSV  1473.47  2392.0   -17.02       X-LC  93.18  129921.0   
18      CAMPUS   294.86   393.0   -10.09       M-SC  57.95  157357.0   
87      VIPIND   488.80   718.0  -290.46       H-SC  45.45   82192.0   
10  BAJFINANCE  7055.29  9150.0   -29.24       X-LC  92.05  222020.0   
13   BATAINDIA  1550.24  2096.0   -27.22       M-SC  54.55  109041.0   

    Current P/L  FTT Amt  Today P/L%  Current P/L%   FTT%   OTT%  XIRR  \
9       14990.0  56659.0       -2.79         13.04  43.61  62.34   8.0   
18     -10418.0  66263.0       -2.66         -6.21  42.11  33.28 -12.0   
87     -13124.0  57814.0       -2.37        -13.77  70.34  46.89 -30.0   
10       3306.0  61633.0       -2.36          1.51  27.76  29.69  15.0   
13     -19629.0  64923.0       -2.22        -15.26  59.54  35.20  -6.0   

    RRR Ind  CurrAlloc%  Gained% Criteria Strategy  Category  
9      0.26        1.13    12.82      X40      BTT   FINANCE  
18    -0.16        1.37    28.93     XY24      NTT  FOOTWEAR  
87    -0.23        0.72     0.00     X40N      NTT      MISC  
10     0.05        1.94    14.05      X40      BTT   FINANCE  
13    -0.30        0.95     2.67      X40      NTT  FOOTWEAR

In [12]:
# top 5 to monitor
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

Symbol  AvgCost      FTT  Dev%_PE Conviction    RSP   Current  \
50      LUXIND  1448.19  2442.00    27.32       M-SC   3.41  160730.0   
82  TTKPRESTIG   769.29  1149.24    13.14       X-SC  40.91  101093.0   
14   BAYERCROP  4346.57  6805.00    12.32       H-SC  11.36   83414.0   
49        LTIM  5664.39  7262.39    10.95       X-LC  23.86  175146.0   
53  METROPOLIS  1399.27  3102.00     8.89       X-SC  14.77  125588.0   

    Current P/L  FTT Amt  Today P/L%  Current P/L%   FTT%    OTT%  XIRR  \
50      23152.0  71252.0        0.20         16.83  44.33   68.62  11.0   
82        316.0  49455.0        0.50          0.31  48.92   49.39   0.0   
14      13869.0  25466.0        0.70         19.94  30.53   56.56  12.0   
49       5214.0  42718.0        1.50          3.07  24.39   28.21   1.0   
53      31837.0  82248.0        0.13         33.96  65.49  121.69  22.0   

    RRR Ind  CurrAlloc%  Gained% Criteria Strategy    Category  
50     0.32        1.40    57.63     X40N      NTT    TEXTILES  
82     0.01        0.88    16.12     X40N      ATH    DURABLES  
14     0.54        0.73     6.79     X40N      NTT   CHEMICALS  
49     0.12        1.53    28.66     X200      ATH          IT  
53     0.39        1.10    20.49       XR      NTT  HEALTHCARE

In [13]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[~df_portfolio_features['Criteria'].isin(['XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

Symbol    AvgCost        FTT  Dev%_PE Conviction    RSP   Current  \
37         IEX     133.39     219.00    -9.11       H-SC  36.36  187623.0   
53  METROPOLIS    1399.27    3102.00     8.89       X-SC  14.77  125588.0   
49        LTIM    5664.39    7262.39    10.95       X-LC  23.86  175146.0   
29       GLAND    1463.94    3442.00    -1.24       H-SC  78.41   90436.0   
54         MRF  112863.40  150508.27   -10.05       X-MC  21.59  113850.0   

    Current P/L  FTT Amt  Today P/L%  Current P/L%    FTT%    OTT%  XIRR  \
37      38360.0  57431.0        0.50         25.70   30.61   64.18  19.0   
53      31837.0  82248.0        0.13         33.96   65.49  121.69  22.0   
49       5214.0  42718.0        1.50          3.07   24.39   28.21   1.0   
29      11383.0  95428.0        0.34         14.40  105.52  135.12  23.0   
54        987.0  36660.0       -0.32          0.87   32.20   33.35   NaN   

    RRR Ind  CurrAlloc%  Gained% Criteria Strategy    Category  
37     0.67        1.64    27.21       XR      NTT        MISC  
53     0.39        1.10    20.49       XR      NTT  HEALTHCARE  
49     0.12        1.53    28.66     X200      ATH          IT  
29     0.12        0.79     4.29       XR      NTT      PHARMA  
54     0.03        0.99     0.75       AR      ATH       TYRES

In [14]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

Symbol  AvgCost     FTT  Dev%_PE Conviction    RSP   Current  Current P/L  \
28  GICRE   335.06   583.0     5.15       M-MC  64.77  180962.0      34541.0   
40   INFY  1436.24  2275.0     8.81       X-LC  87.50  202758.0      53389.0   

    FTT Amt  Today P/L%  Current P/L%   FTT%  OTT%   XIRR  RRR Ind  \
28  73814.0       -2.17         23.59  40.79  74.0  110.0     0.47   
40  33840.0        0.49         35.74  16.69  58.4   37.0     1.58   

    CurrAlloc%  Gained% Criteria Strategy   Category  
28        1.58    38.96     XY24      BTT  INSURANCE  
40        1.77    41.48      X40      BTT         IT

In [15]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

Symbol   AvgCost       FTT  Dev%_PE Conviction     RSP   Current  \
21   COFFEEDAY     59.14     80.00  -129.93       L-SC  100.00   55219.0   
16   BRITANNIA   4858.30   6446.05    -0.22       X-LC   98.86  166010.0   
23       DABUR    518.36    735.00    -8.65       X-MC   97.73  130879.0   
35  HINDUNILVR   2443.56   3513.00   -14.39       X-LC   96.59  149083.0   
36      HONAUT  42646.15  58518.52   -15.61       X-MC   95.45  122573.0   

    Current P/L  FTT Amt  Today P/L%  Current P/L%    FTT%   OTT%   XIRR  \
21     -58330.0  98378.0       -0.00        -51.37  178.16  35.27    NaN   
16        828.0  53156.0        0.42          0.50   32.02  32.68  239.0   
23      -1303.0  56553.0        0.15         -0.99   43.21  41.79   -1.0   
35      -4861.0  72231.0       -0.07         -3.16   48.45  43.77   -4.0   
36      -5365.0  52988.0        0.67         -4.19   43.23  37.22   97.0   

    RRR Ind  CurrAlloc%  Gained% Criteria Strategy    Category  
21    -0.59        0.48    27.14       XR      NTT      HOTELS  
16     0.02        1.45     5.95       AR      ATH        FMCG  
23    -0.02        1.14     4.63     XY24      BTT        FMCG  
35    -0.07        1.30     9.34      X40      BTT        FMCG  
36    -0.10        1.07    12.33     X40N      ATH  ELECTRICAL

In [16]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

Top_N  Sum_Alloc%
0     10       19.51
1     25       40.97
2     50       71.49

In [17]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24    40.23
X40     19.27
XR      13.58
X40N    13.38
AR       7.97
XY25     2.62
X200     1.53
SR       1.41
Name: CurrAlloc%, dtype: float64

In [18]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
M-SC    27.05
X-LC    22.84
H-SC    17.60
M-MC    10.85
X-SC     5.66
X-MC     5.11
H-MC     2.77
L-SC     2.62
M-LC     2.53
H-LC     1.36
Name: CurrAlloc%, dtype: float64

In [19]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

CurrAlloc%  Current P/L%    FTT%
Category                                       
BANKS               13.54        -18.00   74.00
FINANCE             12.38        -15.19   63.70
IT                   8.98         -8.21   62.59
FMCG                 8.85         -6.07   44.50
MISC                 7.41         -6.76   46.33
PAINTS               5.85        -22.54   58.81
CHEMICALS            5.77         -9.51   78.60
HEALTHCARE           4.87          2.12   46.32
INSURANCE            4.29          1.70   51.33
AUTO                 3.30         -9.14   57.13
FOOTWEAR             2.96        -21.09   61.10
POWER                2.24        -10.17   54.23
TRAVEL               2.13        -25.67   76.98
TEXTILES             2.08          3.72   46.85
CERAMICS             2.01        -24.15   74.66
STEEL                1.58        -24.22  116.79
JEWELLERY            1.54        -64.08  115.97
ENGINEERING          1.45         -9.65   51.89
REFINERIES           1.40         -2.52   21.82
CEMENT               1.24        -32.10  100.07
ELECTRICAL           1.07         -4.38   43.23
ENTERTAINMENT        1.06        -33.57  100.73
TYRES                0.99          0.87   32.20
DURABLES             0.88          0.31   48.92
AC                   0.85        -31.23  103.66
PHARMA               0.79         12.59  105.52
HOTELS               0.48       -105.63  178.16

In [20]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Criteria                  
XY24      3045955.0     30
XR        1348740.0     17
X40N       998434.0     16
X40        826798.0     12
AR         719291.0      8
SR         184343.0      2
XY25        71238.0      2
X200        42718.0      1

In [21]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction                  
M-SC        2208255.0     27
H-SC        1741416.0     16
M-MC         948078.0     10
X-LC         866896.0     14
X-SC         463395.0      6
L-SC         296210.0      4
X-MC         243715.0      5
H-MC         150518.0      2
M-LC         119417.0      2
H-LC          79800.0      1

In [22]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
M-SC       XY24      1065865.0     11
M-MC       XY24       739942.0      7
H-SC       XY24       650972.0      4
X-LC       X40        625530.0      9
M-SC       XR         466157.0      6
           X40N       424679.0      6
H-SC       XR         398532.0      5
           X40N       394367.0      6
           AR         297545.0      1
X-SC       XR         188365.0      2
L-SC       XR         163895.0      2
H-MC       XY24       150518.0      2
M-MC       XR         131791.0      2
X-LC       AR         127410.0      2
M-SC       AR         123605.0      2
X-SC       SR         121317.0      1
M-LC       XY24       119417.0      2
X-MC       X40N        93957.0      2
X-SC       X40N        85431.0      2
H-LC       X40         79800.0      1
M-MC       AR          76345.0      1
L-SC       XY24        74589.0      1
X-LC       XY25        71238.0      2
X-SC       XY24        68282.0      1
M-SC       X40         64923.0      1
           SR          63026.0      1
L-SC       AR          57726.0      1
X-MC       XY24        56553.0      1
           X40         56545.0      1
X-LC       X200        42718.0      1
X-MC       AR          36660.0      1

In [23]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 20.0 seconds
